# AspC insertion

In [22]:
%run 1_Objective_function.ipynb

ASPtex ASPtex: asp__L_e <=> asp__L_p L-aspartate transport via diffusion (extracellular to periplasm)
EX_asp__L_e EX_asp__L_e: asp__L_e -->  L-Aspartate exchange
0.87699721425716
0.87392
4.596027557488181


Then, we will add a new reaction that produces L-aspartate from OAA transamination using Pseudomonas aeruginosa's AspC (PaeAspDH). This reaction will be added to the model as a heterologous reaction.

In [11]:
from cobra.io import read_sbml_model

model_pP = read_sbml_model('Models/iJN746.xml')

We know that the aspartate aminotransferase is aspC. To find it will make a list with all the genes that are related to aspartate.

In [18]:
aspartate_related_reactions = [reaction for reaction in model_pP.reactions if 'aspartate' in reaction.name.lower()]
for reaction in aspartate_related_reactions:
    print(reaction.id, reaction.name)
    print("Associated Genes:", [gene.id for gene in reaction.genes])


ASPK Aspartate kinase
Associated Genes: ['PP_4473']
ASPabcpp L-aspartate transport via ABC system (periplasm)
Associated Genes: ['PP_1297', 'PP_1071', 'PP_1299', 'PP_1298', 'PP_1068', 'PP_1300', 'PP_1070', 'PP_1069']
ASPt2_2pp Aspartate transport via proton symport (2 H) (periplasm)
Associated Genes: ['PP_1188']
ASPtex L-aspartate transport via diffusion (extracellular to periplasm)
Associated Genes: []
ASAD Aspartate-semialdehyde dehydrogenase
Associated Genes: ['PP_1989']
ASPCT Aspartate carbamoyltransferase
Associated Genes: ['PP_4998']
ASPO1 L-aspartate oxidase
Associated Genes: ['PP_1426']
ASPO6 L-aspartate oxidase
Associated Genes: ['PP_1426']
ASPTA Aspartate transaminase
Associated Genes: ['PP_3721']
EX_asp__L_e L-Aspartate exchange
Associated Genes: []


We found it! It's the same ASPTA reaction. Therefore the gene is PP_3721. We can now check that this gene is truly responsible for the reaction.

In [31]:
gene_id = 'PP_3721' 

# Find the gene in the model
gene = model_pP.genes.get_by_id(gene_id)

# Print the reactions associated with this gene
for reaction in gene.reactions:
    print("Reaction ID:", reaction.id)
    print("Reaction:", reaction.reaction)
    print("---")


Reaction ID: ARUH
Reaction: arg__L_c + pyr_c <=> 5g2oxpt_c + ala__L_c
---
Reaction ID: ASPTA
Reaction: akg_c + asp__L_c <=> glu__L_c + oaa_c
---


We can see our gene is responsible for more than one reaction. That might end up affecting our flux. To check if the flux changes after the addition of this enzyme, we will now add it to the model.

In [37]:
model = read_sbml_model('Models/iML1515.xml')

In [53]:
new_reaction = Reaction('ASPTA_new2') #Created a new variant of the reaction. Two AspCs will now be present in the cell.
new_reaction.name = 'PpAspC'
new_reaction.subsystem = 'Subsystem'
new_reaction.lower_bound = 0  # Cannot be 0 if the reaction is reversible
new_reaction.upper_bound = 100 

# Add metabolites (reactants and products)
# You need to specify the metabolites and their coefficients (use negative for reactants)
metabolites_dict = {
    model.metabolites.get_by_id('glu__L_c'): -1.0,  # Reactant with coefficient
    model.metabolites.get_by_id('oaa_c'): -1.0,
    model.metabolites.get_by_id('asp__L_c'): 1.0,   # Product with coefficient
    model.metabolites.get_by_id('akg_c'): 1.0,
}
new_reaction.add_metabolites(metabolites_dict)

# Add reaction to the model
model.add_reactions([new_reaction])

After adding the enzyme and reaction, we can simulate the modified E. coli model to assess the impact on aspartate production.

In [56]:
with model:
    model.objective = model.reactions.EX_asp__L_e
    model_asp_optimized = print(model.optimize().objective_value)

17.975172413793075


We also check for the impact on biomass.

In [55]:
model.optimize()
print("Objective value:", model.objective.value)

Objective value: 0.8769972144269985


It does not seem to change the biomass or aspartate. This could be because of a low availability of the substrates or other bottlenecks in the pathway.